In [1]:
import numpy as np
import torch as th
from matplotlib import pyplot as plt
from time import time as get_time
from AIGLE import GLESimulator, LESimulator
from AIGLE import AIGLE
from AIGLE.interpolation import interpolated_fes_2d
from AIGLE.utilities import *

np.set_printoptions(precision=12)
th.set_printoptions(precision=6)


dev = 'cuda' if th.cuda.is_available() else 'cpu'
def mic(x):
    return (x+np.pi) % (2*np.pi) - np.pi
 

## Load FES and create force engine

In [2]:
kbT2kjmol = 2.479
free_energy = np.load('./metadynamics/biases/free_energy.npy') #kJ/mol
free_energy -= free_energy.min()
free_energy /= kbT2kjmol  # in unit of kbT
free_energy = free_energy.T
nbins = [free_energy.shape[0], free_energy.shape[1]]
x0, y0 = -np.pi, -np.pi       ## origin of the free energy surface table
dx = 2*np.pi / (nbins[0]-1)   ## grid size (x-direction)
dy = 2*np.pi / (nbins[1]-1)   ## grid size (y-direction)
interp_fes = interpolated_fes_2d(free_energy, x0,y0,dx,dy, pbc=True, use_torch=False)
print('FES min={}, max={}, unit=kbT'.format(free_energy.min(), free_energy.max()))
energy_upper_bound = None  ##  the trust region of free energy surface


FES min=0.0, max=16.917769247795338, unit=kbT


# simulate GLE

In [ ]:
## load GLE configurations
import json
with open('gle_paras/model_iter_9000.json',) as f:
    gle_config = json.load(f) 

np.random.seed(1234)
## setup systems
dt = 0.0025  #ps
log_freq = 40
Dt = dt * log_freq  # dump position every 0.1ps
mass = np.array(gle_config['mass'])
transform_matrix = np.array(gle_config['transform_matrix'])
x0 = np.array([-2.5,2.5]).reshape(1,2)
x0 = (x0 @ transform_matrix).flatten()

## set up simulator
simulation = GLESimulator(gle_config, timestep=dt, ndim=2, mass = mass)
simulation.set_force_engine(interp_fes.get_force_engine(transform_matrix.T))
simulation.set_energy_engine(interp_fes.get_energy_engine(transform_matrix.T))
simulation.set_position(x0)


In [ ]:
# relax
nrelax = int(10 / dt) # relax 10ps
simulation.step(nrelax, energy_upper_bound)

## simulation
tot_time = 2000 ## ns
nreps = int(tot_time * 1000 / Dt)
t0=get_time()
x_list = []
for idx in range(nreps):
    simulation.step(log_freq, energy_upper_bound)
    x_list.append( simulation.x  * 1.0)
    if idx % int(nreps//20) == 0:
        print('t={:.2f}ns, temp={}, x={}nm, v={}rad/ps, wall time {:.2f}s'.format(
            simulation._step * dt/1000, 
            simulation.get_instant_temp(), 
            simulation.x.reshape(1,2) @ transform_matrix.T, 
            (simulation.p/mass).reshape(1,2) @ transform_matrix.T, 
            get_time()-t0))
        np.save('GLE_DT100fs.npy', np.array(x_list))
print('Done!')

t=0.01ns, temp=5.874775642558887, x=[[-0.815476668163  2.021945924772]]nm, v=[[-32.669540296849   8.138091668408]]rad/ps, wall time 0.00s


## simulate LE

In [ ]:
le_simulation = simulation.get_langevin_integrator(dt)
le_simulation.set_position(x0)

# relax
nrelax = int(10 / dt) # relax 10ps
le_simulation.step(nrelax, energy_upper_bound)

## simulation
tot_time = 2000 ## ns
nreps = int(tot_time * 1000 / Dt)
t0=get_time()
x_list = []
for idx in range(nreps):
    le_simulation.step(log_freq, energy_upper_bound)
    x_list.append( le_simulation.x  * 1.0)
    if idx % int(nreps//20) == 0:
        print('t={:.2f}ns, temp={}, x={}nm, v={}rad/ps, wall time {:.2f}s'.format(
            le_simulation._step * dt/1000, 
            le_simulation.get_instant_temp(), 
            le_simulation.x.reshape(1,2) @ transform_matrix.T, 
            le_simulation.v.reshape(1,2) @ transform_matrix.T, 
            get_time()-t0))
        np.save('LE_DT100fs.npy', np.array(x_list))
print('Done!')